# CNN em Keras para MNIST (Reprodução do Artigo)

Este notebook implementa, em **TensorFlow/Keras**, a CNN descrita no artigo

> *AN OPTIMIZED CONVOLUTIONAL NEURAL NETWORK FOR HANDWRITTEN DIGITAL RECOGNITION CLASSIFICATION*

Aqui fazemos:

1. Preparação do dataset MNIST  
2. Definição de um modelo CNN com profundidade variável (1, 2 ou 3 blocos Conv+Pool)  
3. Treinamento com **5-fold cross-validation**  
4. Experimentos variando:
   - **taxa de aprendizagem (learning rate)**  
   - **número de épocas (epochs)**  
   - **profundidade da rede (número de blocos Conv+Pool)**  
5. Geração de **gráficos** para análise visual dos resultados.

In [ ]:
# Imports principais
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

## 1. Carregando e preparando o dataset MNIST

- Imagens 28x28 em escala de cinza (0–255)  
- Normalizamos para o intervalo **[0, 1]**  
- Adicionamos um eixo de canal para ficar no formato `(28, 28, 1)`  
- Codificamos os rótulos em *one-hot* (10 classes: dígitos de 0 a 9).

In [ ]:
# Carregar MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalização para [0,1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Adicionar eixo de canal (grayscale)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# One-hot encoding dos rótulos
num_classes = 10
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

x_train.shape, y_train_cat.shape

## 2. Definição do modelo CNN (com profundidade variável)

O artigo trabalha com a ideia de **profundidade** da rede como o número de blocos:

> bloco = `Conv2D → ReLU → MaxPooling2D`

Assim, temos:

- **1 bloco** → rede mais rasa  
- **2 blocos** → rede moderadamente profunda (melhor desempenho no artigo)  
- **3 blocos** → rede mais profunda (piora um pouco o desempenho)  

Após os blocos, usamos:

- `Flatten` para achatar o mapa de características em um vetor  
- `Dense(128, ReLU)` para interpretação das features  
- `Dense(10, Softmax)` para classificação em 10 dígitos.

In [ ]:
def create_cnn_model(num_blocks=1, learning_rate=0.01, momentum=0.9):
    """Cria a CNN conforme a descrição do artigo, com num_blocks blocos Conv+Pool."""
    model = Sequential()

    # Primeiro bloco Conv+Pool (com shape de entrada definido)
    model.add(
        Conv2D(
            filters=32,
            kernel_size=(3, 3),
            activation="relu",
            input_shape=(28, 28, 1),
        )
    )
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Blocos adicionais (para profundidade 2 ou 3)
    for _ in range(num_blocks - 1):
        model.add(Conv2D(32, (3, 3), activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))

    # Classificador
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))  # tamanho não especificado no artigo
    model.add(Dense(10, activation="softmax"))

    # Otimizador SGD com momentum (como no artigo)
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum)

    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

# Exemplo: modelo raso com 1 bloco
model_example = create_cnn_model(num_blocks=1)
model_example.summary()

## 3. Funções de treino e avaliação com 5-fold cross-validation

O artigo utiliza **5-fold cross-validation** para:

- Reduzir variância na avaliação  
- Obter **média** e **desvio padrão** da acurácia  

Vamos implementar uma função que:

- Recebe `num_blocks`, `learning_rate`, `epochs`  
- Faz o *split* em 5 folds  
- Treina o modelo em cada fold  
- Retorna acurácia média e desvio padrão em %

In [ ]:
def run_kfold(num_blocks, learning_rate, epochs, batch_size=32, verbose=0):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []

    for train_idx, val_idx in kf.split(x_train):
        x_tr, x_val = x_train[train_idx], x_train[val_idx]
        y_tr, y_val = y_train_cat[train_idx], y_train_cat[val_idx]

        model = create_cnn_model(
            num_blocks=num_blocks,
            learning_rate=learning_rate,
            momentum=0.9,
        )
        model.fit(
            x_tr,
            y_tr,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
        )

        loss, acc = model.evaluate(x_val, y_val, verbose=0)
        accuracies.append(acc * 100.0)

    return float(np.mean(accuracies)), float(np.std(accuracies))

## 4. Experimento 1 – Variando a profundidade da rede

Aqui reproduzimos o experimento de **profundidade (model depth)**:

- Testamos **1, 2 e 3 blocos Conv+Pool**  
- Mantemos:
  - `learning_rate = 0.01`  
  - `epochs = 10` (baseline)  

Em seguida, ploteremos um gráfico de barras com a acurácia média de cada configuração.

In [ ]:
depths = [1, 2, 3]
depth_means = []
depth_stds = []

for d in depths:
    mean_acc, std_acc = run_kfold(num_blocks=d, learning_rate=0.01, epochs=10, verbose=0)
    depth_means.append(mean_acc)
    depth_stds.append(std_acc)
    print(f"Profundidade {d} blocos: média = {mean_acc:.3f}%, desvio = {std_acc:.3f}%")

In [ ]:
# Gráfico: acurácia média x profundidade
plt.figure()
x_pos = np.arange(len(depths))
plt.bar(x_pos, depth_means, yerr=depth_stds)
plt.xticks(x_pos, [str(d) for d in depths])
plt.xlabel("Número de blocos Conv+Pool (profundidade)")
plt.ylabel("Acurácia média (%)")
plt.title("Acurácia vs Profundidade da CNN (5-fold)")
plt.show()

## 5. Experimento 2 – Variando o *learning rate*

Agora avaliamos o impacto do **learning rate**:

- Mantemos:
  - 1 bloco Conv+Pool (modelo baseline)  
  - 10 épocas  
- Testamos os valores usados no artigo:
  - 0.01, 0.05, 0.08, 0.10, 0.15  

Em seguida, ploteremos um gráfico com a acurácia média por valor de learning rate.

In [ ]:
lrs = [0.01, 0.05, 0.08, 0.1, 0.15]
lr_means = []
lr_stds = []

for lr in lrs:
    mean_acc, std_acc = run_kfold(num_blocks=1, learning_rate=lr, epochs=10, verbose=0)
    lr_means.append(mean_acc)
    lr_stds.append(std_acc)
    print(f"LR={lr}: média = {mean_acc:.3f}%, desvio = {std_acc:.3f}%")

In [ ]:
# Gráfico: acurácia média x learning rate
plt.figure()
x_pos = np.arange(len(lrs))
plt.plot(x_pos, lr_means, marker="o")
plt.xticks(x_pos, [str(lr) for lr in lrs])
plt.xlabel("Learning rate")
plt.ylabel("Acurácia média (%)")
plt.title("Acurácia vs Learning rate (5-fold)")
plt.grid(True)
plt.show()

## 6. Experimento 3 – Variando o número de épocas

Agora estudamos o efeito de **mais épocas de treino** na performance:

- Mantemos:
  - 1 bloco Conv+Pool  
  - learning rate = 0.01  
- Testamos:
  - 10, 15, 20, 25, 30 épocas  

Em seguida, veremos o gráfico de acurácia média por número de épocas.

In [ ]:
epochs_list = [10, 15, 20, 25, 30]
ep_means = []
ep_stds = []

for ep in epochs_list:
    mean_acc, std_acc = run_kfold(num_blocks=1, learning_rate=0.01, epochs=ep, verbose=0)
    ep_means.append(mean_acc)
    ep_stds.append(std_acc)
    print(f"Épocas={ep}: média = {mean_acc:.3f}%, desvio = {std_acc:.3f}%")

In [ ]:
# Gráfico: acurácia média x número de épocas
plt.figure()
x_pos = np.arange(len(epochs_list))
plt.plot(x_pos, ep_means, marker="o")
plt.xticks(x_pos, [str(ep) for ep in epochs_list])
plt.xlabel("Número de épocas")
plt.ylabel("Acurácia média (%)")
plt.title("Acurácia vs Número de épocas (5-fold)")
plt.grid(True)
plt.show()

## 7. Conclusões

A partir dos experimentos, você pode comparar com os resultados do artigo:

- **Profundidade**: geralmente, 2 blocos Conv+Pool tendem a performar melhor que 1 ou 3 blocos.  
- **Learning rate**: valores muito altos (ex: 0.1, 0.15) degradam a acurácia; algo próximo a **0.01** costuma ser mais estável.  
- **Número de épocas**: aumentar as épocas nem sempre melhora indefinidamente; há um ponto de retorno decrescente, e o custo computacional cresce.  

Você pode ajustar os hiperparâmetros e repetir os blocos de código dos experimentos para novas análises.